# Global variable

Ce n'est pas idéal, mais c'est une solution temporaire pour choisir tout les paramètres au début de l'analyse sans avoir à les chercher dans les différentes cellules...

In [ ]:
directory = "/media/admin-suz/Datas/Comparative-analysis_FoldsLeg/fold-5/1160/"


SKELETON_NAME     = "20210630_EcadGFP_FastAiSc-1160x1160_px0.09um_dweel0-90_zoom2_002-GB1-fold5.fits_c700.up.NDskl.BRK.a.NDskl"
IMAGE_NAME        = "20210630_EcadGFP_FastAiSc-1160x1160_px0.09um_dweel0-90_zoom2_002-GB1.tif"
IMAGE_NAME_MYOSIN = "20210630_EcadGFP_FastAiSc-1160x1160_px0.09um_dweel0-90_zoom2_002-GB1.tif"

# Pixel/Voxel size
# Airyscan
X_SIZE = 0.1832311/2
Y_SIZE = 0.1832311/2
Z_SIZE = 0.2201818

# Confocal
# X_SIZE = 0.2075665/2
# Y_SIZE = 0.2075665/2
# Z_SIZE = 0.4355092

In [ ]:
# Save face, edge, vert dataframe from initial segmentation
SAVE_INIT_SEGMENTATION = True
# Save face, edge, vert image from initial segmentation
SAVE_INIT_SEGMENTATION_IMAGE = True

SAVE_SIGNAL_MEASURE = True

DIRECT_VISUALISATION_SKELETON = True
DIRECT_VISUALISATION_VERTEX_EDGES = True
DIRECT_VISUALISATION_FACE = True

# Analyse morphologique
MORPHOLOGICAL_ANALYSIS = True
SAVE_MORPHOLOGICAL_ANALYSIS = True
AREA = True
PERIMETER = True
NB_NEIGHBOR = True
ANISO = True
J_ORIENTATION = True
DIRECT_VISUALISATION_ORIENTATION = True
DIRECT_VISUALISATION_SIGNAL = True

INTENSITY_ANALYSIS = True

# Largeur d'épaississement de la surface de la cellule
# Cette valeur est ajoutée au dessus et en dessous du plan de la face
THICKNESS = 0.5
DILATION = 3

SAVE_MORPHOLOGICAL_ANALYSIS = True
SAVE_INTENSITY_ANALYSIS = True

# Import 

In [ ]:
from Dissects.io import (load_NDskl, 
                         load_image,
                         load_skeleton,
                         save_skeleton,
                         save_fits,
                         save_image,
                        )

from Dissects.image import (z_project,
                            thinning,
                            dilation)

from Dissects.geometry import Skeleton

from Dissects.draw.plt_draw import (plot_skeleton_3D,
                                    plot_face_3D,
                                    plot_junction_3D,
                                    plot_aniso_cell,
                                    plot_junction_analyse_3D,
                                    plot_face_analyse_3D
                                   )

from Dissects.analysis.analysis import (general_analysis,
                              cellstats
                              )

from Dissects.analysis.analysis_3D_apical import (junction_intensity,
                                                  face_intensity,
                                                  morphology_analysis)

from Dissects.segmentation.seg_3D_apical import (Segmentation3D)


In [ ]:
import os
import time

from skimage import io
import numpy as np
import pandas as pd
import copy
import sys
from skimage import morphology

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import plotly.graph_objects as go
import pylab as pl

sys.setrecursionlimit(10000)

import plotly.express as px

from scipy.ndimage.morphology import (binary_fill_holes,
                                      binary_dilation,
                                     )
from scipy import ndimage
import scipy as sci
# %matplotlib notebook
%load_ext snakeviz

In [ ]:
import matplotlib as mpl

# update fig parameter
mpl.rcParams['figure.dpi'] = 150 #
mpl.rcParams['figure.figsize'] = (10,10)

# Load Data

In [ ]:
# get original image
tiff_junction = io.imread(os.path.join(directory, 
                              IMAGE_NAME))


In [ ]:
# Create skeleton object
cp, fil, point, cp_fil_info, specs = load_NDskl(os.path.join(directory, SKELETON_NAME))
skeleton = Skeleton(cp, fil, point, cp_fil_info, specs)

In [ ]:
#Clean skeleton
start = time.time()
skeleton.clean()
end = time.time()
print(end-start)

In [ ]:
specs = {"x_size":X_SIZE,
         "y_size":Y_SIZE,
         "z_size":Z_SIZE,
         "x_shape":tiff_junction.shape[2],
         "y_shape":tiff_junction.shape[1],
         "z_shape":tiff_junction.shape[0]}

In [ ]:
if SAVE_INIT_SEGMENTATION_IMAGE :
    image_skeleton = skeleton.create_binary_image(dilation_width=3)

    save_image(image_skeleton, 
               IMAGE_NAME[:-4] + '_output_skeleton.tif', 
               directory, 
               **specs)

In [ ]:
if DIRECT_VISUALISATION_SKELETON:
    fig = plot_skeleton_3D(skeleton, **specs)
    fig.show()


# 3D vertex detection

In [ ]:
segmentation = Segmentation3D(skeleton, specs)

In [ ]:
start = time.time()

segmentation.generate_segmentation(**segmentation.specs)
# face_df, edge_df, vert_df, points_df = generate_segmentation(skel, img0, **{"X_SIZE":X_SIZE,
#                                                                            "Y_SIZE":Y_SIZE,
#                                                                            "Z_SIZE":Z_SIZE})
end= time.time()
print(end-start)

### Save few outputs

In [ ]:
if SAVE_INIT_SEGMENTATION : 
    segmentation.vert_df.to_csv(os.path.join(directory, 'vert_df.csv'))
    segmentation.edge_df.to_csv(os.path.join(directory, 'edge_df.csv'))
    segmentation.face_df.to_csv(os.path.join(directory, 'face_df.csv'))


In [ ]:
if SAVE_INIT_SEGMENTATION_IMAGE :
    vertex_image = segmentation.image_identity_vertex(binary=False, dilation_width=3)
    save_image(vertex_image, 
               IMAGE_NAME[:-4] + '_output_vertex_test.tif', 
               directory, 
               **segmentation.specs)
    
    junction_image = segmentation.image_identity_junction(dilation_width=3, aleatory=True)
    save_image(junction_image, 
               IMAGE_NAME[:-4] + '_output_junction_test.tif', 
               directory, 
               **segmentation.specs)
    
    face_image = segmentation.image_identity_face(aleatory=False, thickness=THICKNESS)
    save_image(face_image, 
               IMAGE_NAME[:-4] + '_output_face_test.tif', 
               directory, 
               **segmentation.specs)
    

In [ ]:
if DIRECT_VISUALISATION_FACE:
    fig = plot_skeleton_3D(segmentation.skeleton,
                          **segmentation.specs)

    fig = plot_face_3D(segmentation,
                       fig,
                      **segmentation.specs)
    fig.show()

In [ ]:
if DIRECT_VISUALISATION_VERTEX_EDGES: 
    fig = plot_skeleton_3D(segmentation.skeleton,
                          **segmentation.specs)

 
    fig = plot_junction_3D(segmentation,
                           fig,
                          **segmentation.specs)
    fig.show()
    


# Analyse

In [ ]:
# morphometry analysis

if MORPHOLOGICAL_ANALYSIS :
    morphology_analysis(segmentation,
                        area=AREA,
                        perimeter=PERIMETER,
                        nb_neighbor=NB_NEIGHBOR,
                        aniso=ANISO,
                        j_orientation=J_ORIENTATION,
                       )

if INTENSITY_ANALYSIS : 
    junction_intensity(tiff_junction, segmentation, DILATION, 'myosin_intensity')
    enlarge_face = face_intensity(tiff_junction, segmentation, THICKNESS, DILATION, 'myosin_intensity')


In [ ]:
if SAVE_MORPHOLOGICAL_ANALYSIS : 
    segmentation.vert_df.to_csv(os.path.join(directory, 'vert_df_morphology.csv'))
    segmentation.edge_df.to_csv(os.path.join(directory, 'edge_df_morphology.csv'))
    segmentation.face_df.to_csv(os.path.join(directory, 'face_df_morphology.csv'))

In [ ]:
if DIRECT_VISUALISATION_ORIENTATION:
    fig = plot_skeleton_3D(segmentation.skeleton,
                          **segmentation.specs)
    
    fig = plot_aniso_cell(segmentation, fig = fig)    

    fig.show()
    

In [ ]:
analyse_face_image = segmentation.image_analyse_face("area")
save_image(analyse_face_image, 
           IMAGE_NAME[:-4] + '_area_test.tif', 
           directory, 
           **segmentation.specs)

aniso_face_image = segmentation.image_aniso()
save_image(aniso_face_image, 
           IMAGE_NAME[:-4] + '_aniso_test.tif', 
           directory, 
           **segmentation.specs)


analyse_junction_image = segmentation.image_analyse_junction("length")
save_image(analyse_junction_image, 
           IMAGE_NAME[:-4] + '_length_test.tif', 
           directory, 
           **segmentation.specs)


In [ ]:
analyse_face_image = segmentation.image_analyse_face("area", border=False)
save_image(analyse_face_image, 
           IMAGE_NAME[:-4] + '_area_wo_border.tif', 
           directory, 
           **segmentation.specs)


analyse_junction_image = segmentation.image_analyse_junction("length", border=False)
save_image(analyse_junction_image, 
           IMAGE_NAME[:-4] + '_length_wo_border_test.tif', 
           directory, 
           **segmentation.specs)

In [ ]:
segmentation.edge_df['orientation_xz_degree'] = segmentation.edge_df['orientation_xz']*180/np.pi
segmentation.edge_df['orientation_xz_degree'] = [180+a if a<0 else a for a in segmentation.edge_df['orientation_xz_degree']]
segmentation.edge_df['orientation_xz_degree'] = [180-a if a>90 else a for a in segmentation.edge_df['orientation_xz_degree']]

analyse_junction_image = segmentation.image_analyse_junction("orientation_xz_degree", normalize=False, border=False)
save_image(analyse_junction_image, 
           IMAGE_NAME[:-4] + '_orientation_xz_wo_border.tif', 
           directory, 
           **segmentation.specs)

In [ ]:
segmentation.edge_df['orientation_yz_degree'] = segmentation.edge_df['orientation_yz']*180/np.pi
segmentation.edge_df['orientation_yz_degree'] = [180+a if a<0 else a for a in segmentation.edge_df['orientation_yz_degree']]
segmentation.edge_df['orientation_yz_degree'] = [180-a if a>90 else a for a in segmentation.edge_df['orientation_yz_degree']]

analyse_junction_image = segmentation.image_analyse_junction("orientation_yz_degree", normalize=False, border=False)
save_image(analyse_junction_image, 
           IMAGE_NAME[:-4] + '_orientation_yz_wo_border.tif', 
           directory, 
           **segmentation.specs)

In [ ]:

fig = plot_skeleton_3D(segmentation.skeleton,
                      **segmentation.specs)

fig = plot_junction_analyse_3D(segmentation, 'length',border=False,)

fig.show()
    

In [ ]:

fig = plot_skeleton_3D(segmentation.skeleton,
                      **segmentation.specs)

fig = plot_face_analyse_3D(segmentation, 'area', normalize_max=255, border = False, fig = fig)    

fig.show()
    